In [2]:
import advent

grid = advent.get_char_grid(3)
grid.shape

(140, 140)

In [3]:
EMP = '.0123456789' # anything that doesn't count as a symbol
DIGITS = '0123456789'
assert grid.shape[0] == grid.shape[1]
GRID_SIZE = grid.shape[0]

def borders_symbol(grid, num_digits: int, i: int, j: int):
    # Checks if number borders symbol.
    # assumption is that at i, j, there is a number of num_digits
    # i is the row (top to bottom), j is the column (left to right)
    # Horizontals
    if j > 0 and grid[i, j-1] not in EMP: return True
    if j < (GRID_SIZE-num_digits) and grid[i, j+num_digits] not in EMP: return True

    if i > 0 and grid[i-1, j] not in EMP: return True
    if i > 0 and num_digits >= 2 and grid[i-1, j+1] not in EMP: return True
    if i > 0 and num_digits >= 3 and grid[i-1, j+2] not in EMP: return True

    if i < (GRID_SIZE-1) and grid[i+1, j] not in EMP: return True
    if i < (GRID_SIZE-1) and num_digits >= 2 and grid[i+1, j+1] not in EMP: return True
    if i < (GRID_SIZE-1) and num_digits >= 3 and grid[i+1, j+2] not in EMP: return True
    # diagonals
    if i > 0 and j > 0 and grid[i-1, j-1] not in EMP: return True
    if i < (GRID_SIZE-1) and j > 0 and grid[i+1, j-1] not in EMP: return True
    if i > 0 and j < (GRID_SIZE-num_digits) and grid[i-1, j+num_digits] not in EMP: return True
    if i < (GRID_SIZE-1) and j < (GRID_SIZE-num_digits) and grid[i+1, j+num_digits] not in EMP: return True
    return False
borders_symbol(grid, 3, 2, 6)


True

In [4]:
def make_digit(arr):
    return int(''.join(arr))

result = 0
for i, line in enumerate(grid):
    for j, c in enumerate(line):
        if j > 0 and grid[i, j-1] in DIGITS: continue # we already did this num
        if grid[i, j] in DIGITS:
            if j < GRID_SIZE-2 and grid[i, j+1] in DIGITS and grid[i, j+2] in DIGITS: num_digits=3
            elif j < GRID_SIZE-1 and grid[i, j+1] in DIGITS: num_digits=2
            else: num_digits=1
            num = make_digit(grid[i,j:(j+num_digits)])
            if borders_symbol(grid, num_digits, i, j): result += num

print(result)


543867


In [9]:
def digit(grid, i, j):
    # Checks if number is digit
    if i < 0 or j < 0 or i >= grid.shape[0] or j >= grid.shape[1]: return False
    return grid[i, j] in DIGITS

def build_digit(grid, i, j, direction=-1):
    # assumes that grid[i, j] is a digit, 'completes' it in a direction
    if direction == -1:
        if digit(grid, i, j-1) and digit(grid, i, j-2): return make_digit(grid[i, (j-2):(j+1)])
        elif digit(grid, i, j-1): return make_digit(grid[i, (j-1):(j+1)])
        else: return make_digit(grid[i, (j):(j+1)])
    elif direction == 1:
        if digit(grid, i, j+1) and digit(grid, i, j+2): return make_digit(grid[i, j:(j+3)])
        elif digit(grid, i, j+1): return make_digit(grid[i, j:(j+2)])
        else: return make_digit(grid[i, (j):(j+1)])


def gear_borders(grid, i, j):
    # The ways a number can border a gear are:
    # top -1/0: in this case 1 does not need to be checked separately, we know theres 1 number on top
        # but for number have to check both -2 and 1
    # top 0/1: in this case we only have to check 2 since we already caught -1/0/1 in the previous step
    # top 0 is empty: in this case -1 and 1 need to be checked separately
    # top 0 is full but -1 and 1 are both empty: simple edge case, 1 number
    # (same as above but for bottom)
    # left (goes left)
    # right (goes right)
    bordering_numbers = []
    if digit(grid, i-1, j-1) and digit(grid, i-1, j):
        if digit(grid, i-1, j+1): bordering_numbers.append(make_digit(grid[i-1, (j-1):(j+2)]))
        elif digit(grid, i-1, j-2): bordering_numbers.append(make_digit(grid[i-1, (j-2):(j+1)]))
        else: bordering_numbers.append(make_digit(grid[i-1, (j-1):(j+1)]))
    elif digit(grid, i-1, j) and digit(grid, i-1, j+1):
        if digit(grid, i-1, j+2): bordering_numbers.append(make_digit(grid[i-1, (j):(j+3)]))
        else: bordering_numbers.append(make_digit(grid[i-1, (j):(j+2)]))
    elif not digit(grid, i-1, j):
        if digit(grid, i-1, j-1): bordering_numbers.append(build_digit(grid, i-1, j-1, -1))
        if digit(grid, i-1, j+1): bordering_numbers.append(build_digit(grid, i-1, j+1, 1))
    elif digit(grid, i-1, j): # we know j-1 and j+1 must be empty
        bordering_numbers.append(make_digit(grid[i-1, j:(j+1)]))
    
    # YES THIS IS A COPY OF THE WHOLE THING BUT WITH i+1 SORRY
    if digit(grid, i+1, j-1) and digit(grid, i+1, j):
        if digit(grid, i+1, j+1): bordering_numbers.append(make_digit(grid[i+1, (j-1):(j+2)]))
        elif digit(grid, i+1, j-2): bordering_numbers.append(make_digit(grid[i+1, (j-2):(j+1)]))
        else: bordering_numbers.append(make_digit(grid[i+1, (j-1):(j+1)]))
    elif digit(grid, i+1, j) and digit(grid, i+1, j+1):
        if digit(grid, i+1, j+2): bordering_numbers.append(make_digit(grid[i+1, (j):(j+3)]))
        else: bordering_numbers.append(make_digit(grid[i+1, (j):(j+2)]))
    elif not digit(grid, i+1, j):
        if digit(grid, i+1, j-1): bordering_numbers.append(build_digit(grid, i+1, j-1, -1))
        if digit(grid, i+1, j+1): bordering_numbers.append(build_digit(grid, i+1, j+1, 1))
    elif digit(grid, i+1, j): # we know j-1 and j+1 must be empty
        bordering_numbers.append(make_digit(grid[i+1, j:(j+1)]))
    
    if digit(grid, i, j-1): bordering_numbers.append(build_digit(grid, i, j-1, -1))
    if digit(grid, i, j+1): bordering_numbers.append(build_digit(grid, i, j+1, 1))
    return bordering_numbers

result = 0
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        if grid[i, j] == '*':
            nums = gear_borders(grid, i, j)
            if len(nums) == 2: result += nums[0] * nums[1]
print(result)

79613331
